<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest
4. Consumer API keys: 
    
    Z22dLMPEOtCL2ayqnPTbFg3sK (API key)
    
    SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj (API secret key)
    
    Access token & access token secret

    14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb (Access token)

    nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313 (Access token secret)

    Read and write (Access level)




In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [238]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'
twitter_archive_master_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_archive_master.csv'
dog_stage_file = '/content/gdrive/My Drive/Colab Notebooks/dog_stage.csv'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [169]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [0]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')


In [0]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 765
Rate limit reached. Sleeping for: 768


In [0]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print('%s %d %s %d' % ('Valid tweet count: ', len(f.readlines()), 'Error tweet count: ', len(error_tweets)))

In [0]:
# Read "tweet_json.txt" file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest

columns = ['tweet_id', 'text', 'retweet_count', 'favorite_count', 'retweeted', 'favorited', 'media_url']
tweets_mini_df = pd.DataFrame(columns = columns)

index = 0
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    for tweet in f:
        tweet_json = json.loads(tweet)
        tweet_id = tweet_json['id']
        text = tweet_json['text']
        retweet_count = tweet_json['retweet_count']
        favorite_count = tweet_json['favorite_count']
        retweeted = tweet_json['retweeted']
        favorited = tweet_json['favorited']
        try:
            media_url = tweet_json['entities']['media'][0]['media_url_https']
        except KeyError as e:
            media_url = ''

        tweets_mini_df.loc[index] = pd.Series({'tweet_id': tweet_id, 
                                        'text': text, 
                                        'retweet_count': retweet_count, 
                                        'favorite_count': favorite_count,
                                        'retweeted': retweeted,
                                        'favorited': favorited,
                                        'media_url': media_url})
        index +=1


### Assessing data

Key Points

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.

* Cleaning includes merging individual pieces of data according to the rules of tidy data.

* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.

* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


#### twitter-archive-enhanced.csv

In [171]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [172]:
tweets_df.query('name == "None"').sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
1717,680191257256136705,NaN,NaN,2015-12-25 01:00:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a sleepy Christmas pupper 11/10 https:/...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680191257...,11,10,None,None,None,pupper,None
990,748705597323898880,NaN,NaN,2016-07-01 02:31:39 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#BarkWeek is getting rather heckin terrifying ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/748705597...,13,10,None,None,None,None,None
1051,742534281772302336,NaN,NaN,2016-06-14 01:49:03 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","For anyone who's wondering, this is what happe...",NaN,NaN,NaN,https://vine.co/v/iLTZmtE1FTB,11,10,None,doggo,None,None,None
1882,674999807681908736,6.747934e+17,4.196984e+09,2015-12-10 17:11:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ok last one of these. I may try to make some m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674999807...,13,10,None,None,None,None,None


In [173]:
tweets_df.query('name == "None"').shape

(745, 17)

#### image_predictions.tsv

In [175]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [176]:
img_pred_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
368,672968025906282496,https://pbs.twimg.com/media/CVbc2V2WsAE3-kn.jpg,1,toy_poodle,0.678046,True,miniature_poodle,0.160273,True,Airedale,0.065649,True
1147,730924654643314689,https://pbs.twimg.com/media/CiTEFjDXAAAqU6I.jpg,1,polecat,0.185382,False,mink,0.105282,False,Newfoundland,0.086241,True
1127,727644517743104000,https://pbs.twimg.com/media/Chkc1BQUoAAa96R.jpg,2,Great_Pyrenees,0.457164,True,kuvasz,0.391710,True,Labrador_retriever,0.094523,True
988,707776935007539200,https://pbs.twimg.com/media/CdKHWimWoAABs08.jpg,1,miniature_pinscher,0.890426,True,toy_terrier,0.051335,True,Chihuahua,0.018015,True
2057,888804989199671297,https://pbs.twimg.com/media/DFWra-3VYAA2piG.jpg,1,golden_retriever,0.469760,True,Labrador_retriever,0.184172,True,English_setter,0.073482,True


In [177]:
list(img_pred_df)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

#### tweets_mini_df

In [178]:
tweets_mini_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339 entries, 0 to 2338
Data columns (total 7 columns):
tweet_id          2339 non-null object
text              2339 non-null object
retweet_count     2339 non-null object
favorite_count    2339 non-null object
retweeted         2339 non-null object
favorited         2339 non-null object
media_url         2339 non-null object
dtypes: object(7)
memory usage: 146.2+ KB


In [179]:
tweets_mini_df.sample(5)

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url
282,838085839343206401,@bragg6of8 @Andy_Pace_ we are still looking fo...,0,147,False,False,
1463,693582294167244802,Personally I'd give him an 11/10. Not sure why...,270,1711,False,False,
1261,709042156699303936,This is Klevin. He's addicted to sandwiches (y...,1724,4936,False,False,https://pbs.twimg.com/media/CdcGBB3WwAAGBuU.jpg
883,758854675097526272,This is Lilli Bee &amp; Honey Bear. Unfortunat...,958,3724,False,False,https://pbs.twimg.com/media/Cof-SuqUsAU8fp-.jpg
900,757354760399941633,"This is Devón (pronounced ""Eric""). He forgot h...",1523,4733,False,False,https://pbs.twimg.com/media/CoKqIndWgAAattd.jpg


In [180]:
tweets_mini_df.isnull().sum()

tweet_id          0
text              0
retweet_count     0
favorite_count    0
retweeted         0
favorited         0
media_url         0
dtype: int64

#### Cleaning data

#### 1. twitter-archive-enhanced.csv
##### 1.1 Data Quality Issues:

1. There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. These need to be removed from the dataset as we only want original tweets (1.1.1)

2. Remove the following columns from the dataset. I don't think that we need these columns for our analysis (1.1.2)

    *  in_reply_to_status_id
    *  in_reply_to_user_id
    *  retweeted_status_id
    *  retweeted_status_user_id
    *  retweeted_status_timestamp
    *  expanded_urls

3. The "timestamp" column is represented as a string and needs to be converted to datetime (1.1.3)
4. Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements (1.1.4)
5. Dog stages are not fully recorded and many of them contain null values or values labeled as "None". We need to scan the tweet text and see if we can extract these values for each row (1.1.5)
6. *** Not Addressed *** There are 745 rows in this dataset where the name is marked as "None". I am not addressing this issue right now since I don't have sufficient data to obtain this information (1.1.6)

##### 1.2 Tidiness Issues:
1. Dog stages are represented in separate columns as a sparse-array matrix which is wasteful and also not scalable in case we identify new stages. These need to be converted to a single column titled "stage". If they are not mutually exclusive (meaning, if there are multiple tweets about the same dog at different points in time where their stages change), we can put them into a separate dataframe for normalization purposes (1.2.1)


#### 2. image-predictions.tsv

##### 2.1 Tidiness Issues:

1. In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence against each and a boolean flag indicating whether the image prediction is actually canine or not. That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. Addressing this tidiness issue will make it easier to fix the data quality issues identified below. We will begin with separating breed predictions to its own dataframe so that we can work on the data quality issues later (2.1.1)

##### 2.2 Data Quality Issues:

1. Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. We will then pick the prediction with the higher percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a single breed) and drop the ones with lower prediction confidence scores (2.2.1)
2. We can then merge this dataset back to the original dataset (2.2.2)
3. After merging, we will drop the following columns as they are no longer needed (2.2.3): 
    *  jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
    *  img_num (don't see how this information will be useful)
    *  p1
    *  p1_conf
    *  p1_dog
    *  p2
    *  p2_conf
    *  p2_dog
    *  p3
    *  p3_conf
    *  p3_dog




#### 3. tweet_mini_df

##### 3.1 Data Quality Issues
1. We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets (3.1.1)

2. We will also drop the "text" column as we no longer need this (3.1.2)


#### 4. Tidiness Issues: combining data from all the datasets

##### 4.1 Tidiness

1. Having addressed the data quality and tidiness issues at the level of individual datasets, we can now merge the data in these datasets into one. Since the data granularity in each of these datasets is at the "tweet_id" level, we will use it to combine the data into a single dataset and persist it to a file (4.1.1)

##### 4.2 Data Quality

1. We will use "inner joins" to merge these datasets thus eliminating any rows for whcih no valid information is available online. For example, there are 17 tweet ids in the original "twitter-archive-enhanced" dataset for which the Twitter API returned errors and these will be eliminated during the merging process. (4.2.1)

In [181]:
# First, I am making clean copies of each of the dataframes and leaving the originals intact

tweets_df_clean = tweets_df.copy()
img_pred_df_clean = img_pred_df.copy()
tweets_mini_df_clean = tweets_mini_df.copy()

print ('%s %d %s %d' %('Original Row Count: ', tweets_df.shape[0], 'Copy Row Count: ', tweets_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', img_pred_df.shape[0], 'Copy Row Count: ', img_pred_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', tweets_mini_df.shape[0], 'Copy Row Count: ', tweets_mini_df_clean.shape[0]))

Original Row Count:  2356 Copy Row Count:  2356
Original Row Count:  2075 Copy Row Count:  2075
Original Row Count:  2339 Copy Row Count:  2339


In [182]:
"""
1.1.1 Define (twitter-archive-enhanced.csv)
There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. 
These need to be removed from the dataset as we only want original tweets
"""

tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].shape[0]

181

In [0]:
"""
1.1.1 Code
"""

tweets_df_clean.drop(tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].index, inplace = True)


In [0]:
"""
1.1.1 Test
Assertion: Original DF Row Count - Cleaned DF Row Count = Retweet Count
"""

assert (tweets_df.shape[0] - tweets_df_clean.shape[0] == tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0])
# print('%s %d %s %d %s %d' % ('Org. Count: ', tweets_df.shape[0], 'New Count: ', tweets_df_clean.shape[0], 'Diff: ',  tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0]))

In [185]:
"""
1.1.2 Define (twitter-archive-enhanced.csv)
Remove the following columns from the dataset:

in_reply_to_status_id
in_reply_to_user_id
retweeted_status_id
retweeted_status_user_id
retweeted_status_timestamp
expanded_urls
"""
tweets_df_clean_drop_cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls']
tweets_df_clean[tweets_df_clean_drop_cols].head()

,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls
0,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...
1,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...
2,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...
3,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...
4,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...


In [0]:
"""
1.1.2 Code
"""
tweets_df_clean.drop(tweets_df_clean_drop_cols, axis = 1, inplace = True)

In [0]:
"""
1.1.2 Test
"""

assert not set(tweets_df_clean_drop_cols).issubset(tweets_df_clean.columns)

In [188]:
""" 
1.1.3 Define (twitter-archive-enhanced.csv)
The "timestamp" column is represented as a string and needs to be converted to datetime
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(8)
memory usage: 203.9+ KB


In [0]:
"""
1.1.3 Code
"""

tweets_df_clean['timestamp'] = pd.to_datetime(tweets_df_clean['timestamp'])

In [190]:
"""
1.1.3 Test
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 203.9+ KB


In [191]:
"""
1.1.4 Define (twitter-archive-enhanced.csv)
Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements
"""
tweets_df_clean.query('timestamp > "08/01/2017"')

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None,None


In [0]:
"""
1.1.4 Code
"""

tweets_df_clean.drop(tweets_df_clean.query('timestamp > "08/01/2017"').index, inplace = True)

In [0]:
"""
1.1.4 Test
"""

assert tweets_df_clean.query('timestamp > "08/01/2017"').shape[0] == 0

In [194]:
"""
1.1.5 & 1.2.1 Define (twitter-archive-enhanced.csv)
Dog stages are represented in separate columns as a sparse-array matrix which is 
wasteful and also not scalable in case we identify new stages. These need to be 
converted to a single column titled "stage". If they are not mutually exclusive 
(meaning, if there are multiple tweets about the same dog at different points in 
time where their stages change), we can put them into a separate dataframe for 
normalization purposes

Dog stages are not fully recorded and many of them contain null values or values labeled as "None". 
We need to scan the tweet text and see if we can extract these values for each row
"""

dog_stages = ['doggo', 'floofer', 'pupper', 'puppo']
[tweets_df_clean[dog_stages] == 'None']

[      doggo  floofer  pupper  puppo
 2      True     True    True   True
 3      True     True    True   True
 4      True     True    True   True
 5      True     True    True   True
 6      True     True    True   True
 7      True     True    True   True
 8      True     True    True   True
 9     False     True    True   True
 10     True     True    True   True
 11     True     True    True   True
 12     True     True    True  False
 13     True     True    True   True
 14     True     True    True  False
 15     True     True    True   True
 16     True     True    True   True
 17     True     True    True   True
 18     True     True    True   True
 20     True     True    True   True
 21     True     True    True   True
 22     True     True    True   True
 23     True     True    True   True
 24     True     True    True   True
 25     True     True    True   True
 26     True     True    True   True
 27     True     True    True   True
 28     True     True    True   True
 

In [0]:
"""
1.1.5 & 1.2.1 Code
In this cleaning exercise, I am going to transpose the "dog_stage columns" to rows first to make the next steps easier
"""

dog_stage_df = pd.melt(tweets_df_clean[['tweet_id', 'text', 'doggo', 'floofer', 'pupper', 'puppo']], id_vars = ['tweet_id', 'text'], var_name = 'dog_stage_col', value_name = 'dog_stage')

# Next, I am assigning the value "doggo" to the column 'dog_stage' in cases where the tweet text has the word "doggo" in it and the 'dog_stage' column has the value "None"
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('doggo ', case = False)].query('dog_stage_col == "doggo" and dog_stage == "None"').index, 'dog_stage'] = 'doggo'

# Next, I am repeating the above for other 'dog_stage' values
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('floofer ', case = False)].query('dog_stage_col == "floofer" and dog_stage == "None"').index, 'dog_stage'] = 'floofer'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('pupper ', case = False)].query('dog_stage_col == "pupper" and dog_stage == "None"').index, 'dog_stage'] = 'pupper'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('puppo ', case = False)].query('dog_stage_col == "puppo" and dog_stage == "None"').index, 'dog_stage'] = 'puppo'


In [196]:
"""
1.1.5 & 1.2.1 Code
The test below confirm that there are multiple dog stages tied to a single tweet id.
This seems to indicate that a dog can go through multiple stages.
Therefore, i am going to retain this as a separate dataset
However, I am going to get rid of all rows where the dog stage is None even after fixing the quality
"""
dog_stage_df[['tweet_id', 'dog_stage']].query('dog_stage != "None"').groupby('tweet_id').filter(lambda x: len(x) > 1).sort_values(by=['tweet_id'])

,tweet_id,dog_stage
933,733109485275860992,doggo
5279,733109485275860992,pupper
883,741067306818797568,doggo
5229,741067306818797568,pupper
779,751583847268179968,doggo
5125,751583847268179968,pupper
720,759793422261743616,doggo
5066,759793422261743616,pupper
591,781308096455073793,doggo
4937,781308096455073793,pupper


In [197]:
"""
1.1.5 & 1.2.1 Code
We will begin with dropping the unnecessary columns
"""
dog_stage_df.drop(['text', 'dog_stage_col'], axis = 1, inplace = True)
dog_stage_df.info()

"""
Next, we will drop all rows where the dog_stage column contains the value "None"
"""

dog_stage_df.drop(dog_stage_df.query('dog_stage == "None"').index, inplace = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 2 columns):
tweet_id     8692 non-null int64
dog_stage    8692 non-null object
dtypes: int64(1), object(1)
memory usage: 135.9+ KB


In [0]:
"""
1.1.5 & 1.2.1 Test 1
"""

assert dog_stage_df.query('dog_stage == "None"').shape[0] == 0

In [0]:
"""
1.1.5 & 1.2.1 Code
We can now drop the individual dog stage columns from the "tweets_df_clean" dataset
"""
tweets_df_clean.drop(dog_stages, axis = 1, inplace = True)

In [0]:
"""
1.1.5 & 1.2.1 Test 2
"""

assert not set(dog_stages).issubset(tweets_df_clean.columns)

In [203]:
"""
2.1.1 Define (image-predictions.tsv)
In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. 
There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence 
against each and a boolean flag indicating whether the image prediction is actually canine or not. 
That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. 
Addressing this tidiness issue will make it easier to fix the data quality issues identified below. 
We will begin with separating breed predictions to its own dataframe so that we can work on the data quality issues later
"""

img_pred_df_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
143,668620235289837568,https://pbs.twimg.com/media/CUdqjvAWUAANfoU.jpg,1,crash_helmet,0.757942,False,toaster,0.037497,False,mouse,0.027271,False
2049,887101392804085760,https://pbs.twimg.com/media/DE-eAq6UwAA-jaE.jpg,1,Samoyed,0.733942,True,Eskimo_dog,0.035029,True,Staffordshire_bullterrier,0.029705,True
1088,718971898235854848,https://pbs.twimg.com/media/CfpNGTHUIAAA8XC.jpg,1,golden_retriever,0.140394,True,Saint_Bernard,0.118769,True,Labrador_retriever,0.075492,True
253,670727704916926465,https://pbs.twimg.com/media/CU7nSZEW4AA6r5u.jpg,1,wood_rabbit,0.368562,False,tabby,0.309675,False,Egyptian_cat,0.154914,False
1388,766313316352462849,https://pbs.twimg.com/media/CqJ95SRWgAATPK_.jpg,1,toy_poodle,0.966896,True,miniature_poodle,0.016424,True,cocker_spaniel,0.010227,True


In [0]:
"""
2.1.1 Code
"""

img_pred_df_p1 = img_pred_df_clean[['tweet_id', 'p1', 'p1_conf', 'p1_dog']].copy()
img_pred_df_p1.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_p2 = img_pred_df_clean[['tweet_id', 'p2', 'p2_conf', 'p2_dog']].copy()
img_pred_df_p2.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_p3 = img_pred_df_clean[['tweet_id', 'p3', 'p3_conf', 'p3_dog']].copy()
img_pred_df_p3.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_breed = img_pred_df_p1.append(img_pred_df_p2.append(img_pred_df_p3)).copy()


In [205]:
"""
2.1.1 Test
"""

img_pred_df_breed.sort_values(by = 'tweet_id').head(5)


,tweet_id,breed,prediction_confidence,is_canine
0,666020888022790149,Welsh_springer_spaniel,0.465074,True
0,666020888022790149,Shetland_sheepdog,0.061428,True
0,666020888022790149,collie,0.156665,True
1,666029285002620928,redbone,0.506826,True
1,666029285002620928,Rhodesian_ridgeback,0.072010,True


In [206]:
"""
2.2.1 Define  (image-predictions.tsv)
Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. 
We will then pick the prediction with the highest percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a 
single breed) and drop the ones with lower prediction confidence scores

"""
img_pred_df_breed.sort_values(['tweet_id', 'prediction_confidence'], ascending = [True, False]).head(10)

,tweet_id,breed,prediction_confidence,is_canine
0,666020888022790149,Welsh_springer_spaniel,0.465074,True
0,666020888022790149,collie,0.156665,True
0,666020888022790149,Shetland_sheepdog,0.061428,True
1,666029285002620928,redbone,0.506826,True
1,666029285002620928,miniature_pinscher,0.074192,True
1,666029285002620928,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,German_shepherd,0.596461,True
2,666033412701032449,malinois,0.138584,True
2,666033412701032449,bloodhound,0.116197,True
3,666044226329800704,Rhodesian_ridgeback,0.408143,True


In [0]:
"""
2.2.1 Code
"""
img_pred_df_breed = img_pred_df_breed.query('is_canine')
img_pred_df_breed = img_pred_df_breed[img_pred_df_breed.groupby(['tweet_id'])['prediction_confidence'].transform(max) == img_pred_df_breed['prediction_confidence']]


In [0]:
"""
2.2.1 Test
"""
assert img_pred_df_breed.groupby('tweet_id').filter(lambda x: len(x) > 1).shape[0] == 0

In [209]:
"""
2.2.2 Define (image-predictions.tsv)
We can then merge this dataset back to the original dataset

"""
img_pred_df_clean.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [0]:
"""
2.2.2 Code
"""

img_pred_df_clean = img_pred_df_clean.merge(img_pred_df_breed, on = 'tweet_id', how = 'left')

In [211]:
"""
2.2.2 Test
"""
# assert set(['breed', 'prediction_confidence', 'is_canine']).issubset(img_pred_df_clean.columns)
list(img_pred_df_clean)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'breed',
 'prediction_confidence',
 'is_canine']

In [0]:
"""
2.2.3 Define (image-predictions.tsv)
After merging, we will drop the following columns as they are no longer needed:

* jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
* img_num (don't see how this information will be useful)
* p1
* p1_conf
* p1_dog
* p2
* p2_conf
* p2_dog
* p3
* p3_conf
* p3_dog

"""

img_pred_df_clean_dropped_columns = ['jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'is_canine']

In [0]:
"""
2.2.3 Code
"""
img_pred_df_clean.drop(img_pred_df_clean_dropped_columns, axis = 1, inplace = True)

In [214]:
"""
2.2.3 Test
"""

assert not set(img_pred_df_clean_dropped_columns).issubset(img_pred_df_clean.columns)
img_pred_df_clean.head()

,tweet_id,breed,prediction_confidence
0,666020888022790149,Welsh_springer_spaniel,0.465074
1,666029285002620928,redbone,0.506826
2,666033412701032449,German_shepherd,0.596461
3,666044226329800704,Rhodesian_ridgeback,0.408143
4,666049248165822465,miniature_pinscher,0.560311


In [215]:
"""
3.1.1 Define (tweets_mini_df)
We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets
"""

tweets_mini_df_clean = tweets_mini_df.copy()

retweets = tweets_df[tweets_df['retweeted_status_id'].notnull()]['tweet_id'].to_frame(name = 'tweet_id')
retweets.head()

,tweet_id
19,888202515573088257
32,886054160059072513
36,885311592912609280
68,879130579576475649
73,878404777348136964


In [0]:
"""
3.1.1 Code
"""

tweets_mini_df_clean = tweets_mini_df_clean.merge(retweets, on = 'tweet_id', how = 'left', indicator = True)
tweets_mini_df_clean.drop(tweets_mini_df_clean.query('_merge == "both"').index, inplace = True)
tweets_mini_df_clean.drop(['_merge'], axis = 1, inplace = True)


In [217]:
"""
3.1.1 Test
"""

tweets_mini_df_clean.merge(retweets, on = 'tweet_id', how = 'inner')

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url


In [0]:
"""
3.1.2 Define
We will also drop the "text", "retweeted" and "favorited" columns as we no longer need this (3.1.2)
"""
tweets_mini_df_drop_cols = ['text', 'retweeted', 'favorited']

In [0]:
"""
3.1.2 Code
"""

tweets_mini_df_clean.drop(tweets_mini_df_drop_cols, axis = 1, inplace = True)

In [0]:
"""
3.1.2. Test
"""
assert not set(tweets_mini_df_drop_cols).issubset(tweets_mini_df_clean.columns)

In [221]:
"""
4. Define

Having addressed the data quality and tidiness issues at the level of individual datasets, 
we can now merge the data in these datasets into one. Since the data granularity in each 
of these datasets is at the "tweet_id" level, we will use it to combine the data into a 
single dataset and persist it to a file called "twitter_archive_master.csv". 

We will use "inner joins" to merge these datasets thus eliminating any rows for whcih 
no valid information is available online. For example, there are 17 tweet ids in the 
original "twitter-archive-enhanced" dataset for which the Twitter API returned errors 
and these will be eliminated during the merging process.

Inputs:
 - tweets_df_clean
 - img_pred_df_clean
 - tweets_mini_df_clean

Merged into:
 - twitter_archive_master_df
 
Additional files:
 - Since dog stages seem to have a one-to-many relationship with tweets, they will be 
   stored in their own file

"""


'\n4. Define\n\nHaving addressed the data quality and tidiness issues at the level of individual datasets, \nwe can now merge the data in these datasets into one. Since the data granularity in each \nof these datasets is at the "tweet_id" level, we will use it to combine the data into a \nsingle dataset and persist it to a file called "twitter_archive_master.csv". \n\nWe will use "inner joins" to merge these datasets thus eliminating any rows for whcih \nno valid information is available online. For example, there are 17 tweet ids in the \noriginal "twitter-archive-enhanced" dataset for which the Twitter API returned errors \nand these will be eliminated during the merging process.\n\nInputs:\n - tweets_df_clean\n - img_pred_df_clean\n - tweets_mini_df_clean\n\nMerged into:\n - twitter_archive_master_df\n \nAdditional files:\n - Since dog stages seem to have a one-to-many relationship with tweets, they will be \n   stored in their own file\n\n'

In [0]:
"""
4. Code
"""
twitter_archive_master_df = tweets_df_clean.merge(img_pred_df_clean, on = 'tweet_id', how = 'inner').merge(tweets_mini_df_clean, on = 'tweet_id', how = 'inner')


In [234]:
"""
4. Test
"""

twitter_archive_master_df.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,breed,prediction_confidence,retweet_count,favorite_count,media_url
0,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,Chihuahua,0.716012,4023,24408,
1,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,Labrador_retriever,0.168086,8369,41060,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg
2,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,basset,0.555712,9082,39252,
3,891087950875897856,2017-07-29 00:08:17,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,13,10,None,Chesapeake_Bay_retriever,0.425595,3008,19740,
4,890971913173991426,2017-07-28 16:27:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,13,10,Jax,Appenzeller,0.341703,1991,11543,


In [0]:
# Write the dataframes to files
twitter_archive_master_df.to_csv(twitter_archive_master_file)


In [0]:
assert os.path.isfile(twitter_archive_master_file)

In [0]:
dog_stage_df.to_csv(dog_stage_file)


In [0]:
assert os.path.isfile(dog_stage_file)